In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import random

import numpy as np
import torch
import torch_geometric as pyg
from tqdm.auto import *

import deepgd as dgd

In [3]:
device = "cpu"
for backend, device_name in {
    torch.backends.mps: "mps",
    torch.cuda: "cuda",
}.items():
    if backend.is_available():
        device = device_name

In [4]:
batch_size = 128
lr = 0.0005

In [5]:
dataset = dgd.RomeDataset()
model = dgd.DeepGD().to(device)
criteria = {
    dgd.Stress(): 1,
    # dgd.EdgeVar(): 0,
    # dgd.Occlusion(): 0,
    # dgd.IncidentAngle(): 0,
    # dgd.TSNEScore(): 0,
}
optim = torch.optim.AdamW(model.parameters(), lr=lr)

Extracting data/Rome/raw/rome-graphml.tgz
Processing...


Loading graphs:   0%|          | 0/11534 [00:00<?, ?it/s]

Done!


In [6]:
datalist = list(dataset)
random.seed(12345)
random.shuffle(datalist)

In [7]:
train_loader = pyg.loader.DataLoader(datalist[:10000], batch_size=batch_size, shuffle=True)
val_loader = pyg.loader.DataLoader(datalist[11000:], batch_size=batch_size, shuffle=False)
test_loader = pyg.loader.DataLoader(datalist[10000:11000], batch_size=batch_size, shuffle=False)

In [8]:
for epoch in range(1000):
    model.train()
    losses = []
    for batch in tqdm(train_loader):
        batch = batch.to(device)
        model.zero_grad()
        loss = 0
        for c, w in criteria.items():
            loss += w * c(model(batch), batch)
        loss.backward()
        optim.step()
        losses.append(loss.item())
    print(f'[Epoch {epoch}] Train Loss: {np.mean(losses)}')
    with torch.no_grad():
        model.eval()
        losses = []
        for batch in tqdm(val_loader, disable=True):
            batch = batch.to(device)
            loss = 0
            for c, w in criteria.items():
                loss += w * c(model(batch), batch)
            losses.append(loss.item())
        print(f'[Epoch {epoch}] Val Loss: {np.mean(losses)}')

  0%|          | 0/79 [00:00<?, ?it/s]

TypeError: propagate() got an unexpected keyword argument 'edge_weight'